In [6]:
import json
import requests
import json
import speech_recognition as sr
import os
from pydub import AudioSegment
from transformers import pipeline
import numpy as np


/home/usou/venv/superbad/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def converto_to_wav(input_path, output_path = None):
    """
    오디오 파일을 WAV로 변환하는 함수
    
    :param input_path: 변환할 원본 오디오 파일 경로
    :param output_path: 변환된 WAV 파일 경로 (기본적으로 동일한 폴더에 저장)
    :return: 변환된 WAV 파일 경로
    """
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다. {input_path}")
    
    # 출력 경로 설정    
    if output_path is None:
        output_path = os.path.splitext(input_path)[0] + ".wav"
    
    #오디오 파일 로드 및 WAV로 변환
    audio = AudioSegment.from_file(input_path)
    audio.export(output_path, format = "wav")

    print(f"변환 완료: {output_path}")
    return output_path

# ETRI API 키 입력
API_KEY = "41a2a58b-2dae-4d98-b3cc-70a7f0d57887"

# ETRI API 엔드포인트
URL = "http://aiopen.etri.re.kr:8000/WiseNLU"

# 음성을 텍스트로 변환하는 함수
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    
    if not os.path.exists(audio_file):
        return "파일이 존재하지 않습니다."

    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data, language="ko-KR")
        return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없습니다."
    except sr.RequestError as e:
        return f"Google API 요청 오류: {e}"

# 텍스트 감정 분석 요청 함수    
def analyze_emotion(text):
    request_body = {
        "access_key": API_KEY,
        "argument": {
            "text": text,
            "analysis_code": "SA"
        }
    }
    response = requests.post(URL, json=request_body)
    
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API 요청 실패 (HTTP {response.status_code})"}

# 음성 파일 입력
audio_file = "/home/usou/dev_ws/superbad/deeplearning-repo-3/data/usou/test_voice_record/test_1.wav"  # 분석할 음성 파일

# 음성을 텍스트로 변환
recognized_text = speech_to_text(converto_to_wav(audio_file))
print(f"인식된 텍스트: {recognized_text}")

# 오류 메시지가 아닌 경우 감정 분석 실행
# if recognized_text and "오류" not in recognized_text and "없습니다" not in recognized_text:
#     sentiment_result = analyze_emotion(recognized_text)
#     print(json.dumps(sentiment_result, indent=4, ensure_ascii=False))
# else:
#     print("텍스트 변환 실패로 인해 감정 분석을 수행하지 않습니다.")


# 텍스트를 수어체로 변환



변환 완료: /home/usou/dev_ws/superbad/deeplearning-repo-3/data/usou/test_voice_record/test_1.wav
인식된 텍스트: 좋아요


# 형태소 분석 테스트 코드

In [5]:
import requests
import json

# ETRI API 키
API_KEY = "41a2a58b-2dae-4d98-b3cc-70a7f0d57887"  # 본인의 키 입력

# ETRI 형태소 분석 API 엔드포인트
URL = "http://aiopen.etri.re.kr:8000/WiseNLU"

# 요청 데이터
request_body = {
    "access_key": API_KEY,
    "argument": {
        "text": "오늘 날씨가 좋네요.",  # 테스트 문장
        "analysis_code": "morp"  # 형태소 분석 코드
    }
}

# API 요청 실행
response = requests.post(URL, json=request_body)

# 결과 출력
print("형태소 분석 API 응답 코드:", response.status_code)
print("형태소 분석 API 응답 결과:", response.json())


형태소 분석 API 응답 코드: 403
형태소 분석 API 응답 결과: {'result': -1, 'reason': 'Empty Auth Header'}


In [1]:
import pyaudio
import wave
import whisper
from transformers import pipeline
import numpy as np

# 마이크 설정
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000  # Whisper 모델과 호환되는 샘플 레이트
CHUNK = 1024
RECORD_SECONDS = 2  # 녹음 시간
WAVE_OUTPUT_FILENAME = "recorded_audio.wav"

# 감정 분석 모델 로드 (Hugging Face)
emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

def record_audio():
    """마이크에서 음성을 녹음하고 파일로 저장"""
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    print("🎤 음성 녹음 시작... ({}초 동안)".format(RECORD_SECONDS))
    
    frames = []
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("🔴 녹음 종료")

    # 스트림 닫기
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # WAV 파일로 저장
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    return WAVE_OUTPUT_FILENAME

def transcribe_audio(file_path):
    """녹음된 오디오 파일을 Whisper를 이용해 텍스트로 변환"""
    model = whisper.load_model("small")  # Whisper 모델 (tiny, base, small, medium, large 선택 가능)
    result = model.transcribe(file_path)
    return result["text"]

def analyze_emotion(text):
    """텍스트를 감정 분석 모델에 입력하여 감정을 분석"""
    analysis = emotion_model(text)
    return analysis

# 실행 흐름
if __name__ == "__main__":
    audio_file = record_audio()
    transcribed_text = transcribe_audio(audio_file)
    print(f"\n📝 STT 변환 결과: {transcribed_text}")

    emotion_result = analyze_emotion(transcribed_text)
    print("\n📊 감정 분석 결과:")
    for emotion in emotion_result[0]:  # 가장 높은 감정 선택
        print(f"  {emotion['label']}: {emotion['score']:.4f}")

/home/usou/venv/superbad/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0
/home/usou/venv/superbad/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable

🎤 음성 녹음 시작... (2초 동안)
🔴 녹음 종료

📝 STT 변환 결과:  Уitten!

📊 감정 분석 결과:
  anger: 0.2821
  disgust: 0.0263
  fear: 0.0224
  joy: 0.0748
  neutral: 0.1858
  sadness: 0.0226
  surprise: 0.3861


# STT 및 감정 분석

In [7]:
import json
import requests
import json
import speech_recognition as sr
import os
from pydub import AudioSegment
from transformers import pipeline
import numpy as np

# ETRI API 키 입력
API_KEY = "41a2a58b-2dae-4d98-b3cc-70a7f0d57887"

# 감정 분석 모델 로드 (Hugging Face)
emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# ETRI API 엔드포인트
URL = "http://aiopen.etri.re.kr:8000/WiseNLU"


def converto_to_wav(input_path, output_path = None):
    """
    오디오 파일을 WAV로 변환하는 함수
    
    :param input_path: 변환할 원본 오디오 파일 경로
    :param output_path: 변환된 WAV 파일 경로 (기본적으로 동일한 폴더에 저장)
    :return: 변환된 WAV 파일 경로
    """
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다. {input_path}")
    
    # 출력 경로 설정    
    if output_path is None:
        output_path = os.path.splitext(input_path)[0] + ".wav"
    
    #오디오 파일 로드 및 WAV로 변환
    audio = AudioSegment.from_file(input_path)
    audio.export(output_path, format = "wav")

    print(f"변환 완료: {output_path}")
    return output_path

# 음성을 텍스트로 변환하는 함수
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    
    if not os.path.exists(audio_file):
        return "파일이 존재하지 않습니다."

    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data, language="ko-KR")
        return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없습니다."
    except sr.RequestError as e:
        return f"Google API 요청 오류: {e}"
    
    
# # 텍스트 감정 분석 요청 함수        
def analyze_emotion(text):
    """텍스트를 감정 분석 모델에 입력하여 감정을 분석"""
    analysis = emotion_model(text)
    return analysis

# 음성 파일 입력
audio_file = "/home/usou/dev_ws/superbad/deeplearning-repo-3/data/usou/test_voice_record/test_2.wav"  # 분석할 음성 파일

# 음성을 텍스트로 변환
recognized_text = speech_to_text(converto_to_wav(audio_file))
print(f"인식된 텍스트: {recognized_text}")

# 감정 분석 결과
emotion_result = analyze_emotion(recognized_text)
print("\n 감정 분석 결과:")
best_emotion_score = 0
best_emotion = 0
for emotion in emotion_result[0]:  # 가장 높은 감정 선택
    print(f"  {emotion['label']}: {emotion['score']:.4f}")
    best_emotion_score = emotion["score"]

    # if emotion["score"] >= best_emotion_score:
    #     best_emotion = emotion["label"]
    #     print(f"현재 감정 상태는 : {best_emotion}")

    # else:
    #     continue



Device set to use cuda:0


변환 완료: /home/usou/dev_ws/superbad/deeplearning-repo-3/data/usou/test_voice_record/test_2.wav
인식된 텍스트: 50분 알람

 감정 분석 결과:
  anger: 0.0186
  disgust: 0.0104
  fear: 0.0230
  joy: 0.0106
  neutral: 0.8688
  sadness: 0.0282
  surprise: 0.0403
